In [0]:
dbutils.widgets.text("catalog_name", "", "Catalog Name")
dbutils.widgets.text("schema_name", "", "Schema Name")

In [0]:
%sql
USE CATALOG IDENTIFIER(:catalog_name);
USE SCHEMA IDENTIFIER(:schema_name);

--create a fact table
create or replace table fact (sk int, meas int);
insert into fact 
    values  (1, 1)
    ,       (1, 1)
    ,       (1, 1)
    ,       (2, 1)
    ,       (2, 1)
    ,       (2, 1)
    ,       (2, 1)
    ,       (3, 1)
    ,       (3, 1)
    ,       (3, 1)
    ,       (3, 1)
    ,       (3, 1)
    ,       (3, 1);


--create a dim table with a column called "source_1" this wont cause issues
create or replace table dim (sk int, source_1 string, att string);

insert into dim
    values  (1, 'dummy', 'cat')
    ,       (2, 'dummy', 'dog')
    ,       (3, 'dummy', 'rat')
    ,       (4, 'dummy', 'hat');

--create a dim table with a column called "source" this will cause issues
create or replace table dim_error (sk int, source string, att string);
insert into dim_error 
    values  (1, 'dummy', 'cat')
    ,       (2, 'dummy', 'dog')
    ,       (3, 'dummy', 'rat')
    ,       (4, 'dummy', 'hat');

## this works

In [0]:
%sql
CREATE OR REPLACE VIEW source_issue_repro
  WITH METRICS
  LANGUAGE YAML
  AS $$
  version: 0.1

  source: jack_demos.energy.fact

  joins:
    - name: dim2
      source: jack_demos.energy.dim
      on: source.sk = dim2.sk

  dimensions:
    - name: att
      expr: dim2.att
    
  measures:
    - name: Sum of Measure
      expr: SUM(meas)
  $$;




## this doesnt work

In [0]:
%sql
CREATE OR REPLACE VIEW source_issue_repro
  WITH METRICS
  LANGUAGE YAML
  AS $$
  version: 0.1

  source: jack_demos.energy.fact

  joins:
    - name: dim2
      source: jack_demos.energy.dim_error
      on: source.sk = dim2.sk

  dimensions:
    - name: att
      expr: dim2.att
    
  measures:
    - name: Sum of Measure
      expr: SUM(meas)
  $$;


